In [16]:
#start spark local server
import sys, os
from operator import add
import time
import matplotlib.pyplot as plt
import seaborn as sns
os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

c:\Users\namo\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\bezier.py:181: RuntimeWarning: coroutine 'Gdelt2.download_with_dates' was never awaited
  class BezierSegment:


In [18]:
from datetime import date, timedelta
import pandas as pd
import os
import urllib.request
import io

# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
      print(filename)
      if not os.path.exists(filename):
        ## list of countries to look for 
        df_list=[]
        list_countres=['unitedstates']
        queryURLbase = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=(%22climate%20change%22%20OR%20%22global%20warming%22)"
        for country in list_countres:
          queryURLbase= queryURLbase+"%20sourcecountry:"+country+"%20sourcelang:eng&maxrecords=250&mode=artlist&sort=hybridrel"
          print(queryURLbase)
          date = filename.split("_")[0]
          startdate = date+"000000" #0 hour 0 min 0 sec (start of day)
          enddate = date+"235959" #23 hour 59 min 59 sec (end of day)
          query = queryURLbase + "&startdatetime=" + startdate + "&enddatetime=" + enddate
          data = io.StringIO(urllib.request.urlopen(query).read().decode())
          df = pd.read_csv(data, sep=",")
          df['country']=country
         
          
          df_list.append(df)
        df_full=pd.concat(df_list)
        df_full.to_csv(filename)
    except:
        print("Error occurred")


daterange_after = sc.parallelize(pd.date_range('2020 Mar 1','2020 May 24'))
daterange_before = sc.parallelize(pd.date_range('2019 Dec 1','2020 Feb 28'))

dates_before = daterange_before.map(get_filename)
dates_after = daterange_after.map(get_filename)

dates_before.foreach(intofile)
dates_after.foreach(intofile)

c:\Users\namo\AppData\Local\Programs\Python\Python311\Lib\ast.py:276: RuntimeWarning: coroutine 'Gdelt2.download_with_dates' was never awaited
  for item in field:


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 0.0 failed 1 times, most recent failure: Lost task 11.0 in stage 0.0 (TID 11) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:92)
	at org.